<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Concentration_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Concentration calculation
This notebook is used to calculate the concentration of phtalates in an environemental sample using an internal standard for quantitation. The Calibration_calculation.ipynb notebook, yields severall of the required files nescessary for the calculations in this notebook. The files *Cal_{n}_analyte_linreg.csv* need to be used which is located in the *calibration_output* folder on the google drive. If this notebook is not applied from my machine in connection with my google drive, either the path to the required files needs to be adabted or changed.
The following code allows for the calculation of the concentration, by using the linear regression form a calibration series and the ratio of the signal of the analyte with the internal standard (area). All the data regarding the calibration are present on my google drive in the folders: *calibration_input* and *calibration_output*, while a new google drive folder will be used to save the final data in as well as a new input folder.

*Google drive folder:*
quantitation_input
quantitation_output

The .csv files containing the peak list from the sample should be saved in the in the input folder. The names should follow a consequent naming sceme:

***Sample_{date_of_analysis}_{tag_bonus_info}_{Sample_Nr}_pl.csv***

Here the Sample number *(Sample_Nr)* is of key importance and needs to be properly documented in the Labjournal, as it is the main way of differentiation.


The following code is used to import all the nescessary libraries and mount the google drive.

In [1]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import os
import csv
from datetime import datetime
import re
import ast

drive.mount('/content/drive', force_remount = True)

sample_nr_low = 1 #needs to be addapted to 1 nr in the series
sample_nr_high = 4 #needs to be addapted to the last nr of the series

Mounted at /content/drive


The following code is the addapted code from the Calibration_calculation.ipynb and yields the separated in separated .csv files, which will be used to calculate the ratio in a following code segment.

The output file has the following filename and is saved in the quantitation_output folder:

***Sample_{n}_row{i+1}.csv***

In [2]:
#Loop over all the designated sample files.
for n in range(sample_nr_low, sample_nr_high):
    files = glob.glob(f'/content/drive/My Drive/quantitation_input/*_{n}_pl.csv')

    if not files:
        print(f'Files matching pattern Sample Nr. {n} was not found.')
        continue

    else:
        # Loop over each row and save a separate data frame for each
        df = pd.read_csv(files[0], header=4)
        num_rows = len(df)

        for i in range(num_rows):
            row_dfs = []
            for file in files:
                df = pd.read_csv(file, header=4, na_values='')

                # Extract the i'th row of the CSV file and append it to the row data frame list
                row = df.iloc[i, :].dropna().astype(str)
                row_dfs.append(row)

            # Concatenate the row data frames along the columns axis and convert to a data frame
            combined_df = pd.concat(row_dfs, axis=1).T

            # Save the combined data frame to a CSV file with a name indicating the row number
            combined_df.to_csv(f'/content/drive/My Drive/quantitation_output/Sample_{n}_row{i+1}.csv', index=False, header=False)

            # Convert all values to strings before joining and print the row in the desired format
            formatted_row = '\t'.join(map(str, combined_df.values[0]))
            print(formatted_row)

13.84	13.79	13.9	14395275428.047	100.0	7362407851.786	100.0
15.78	15.73	15.83	15750319878.906	14.16	7946363587.808	15.56
18.63	18.57	18.7	6389608124.585	100.0	3088648250.757	100.0
20.64	20.59	20.7	27890441660.95	25.07	13976431665.592	27.36
25.64	25.59	25.7	13499461401.306	12.13	6592637592.24	12.91
26.78	26.73	26.85	24156282005.648	21.71	11754747323.446	23.01
29.32	29.25	29.42	29960227520.599	26.93	10812154667.372	21.17
13.84	13.79	13.9	1567121334.959	100.0	772794172.544	100.0
15.77	15.72	15.84	1684353681.704	19.77	842785059.057	20.52
18.63	18.58	18.68	537221507.941	100.0	265870495.185	100.0
20.64	20.59	20.7	2584870976.683	30.35	1359365659.623	33.1
25.64	25.59	25.71	943856948.752	11.08	478486206.576	11.65
26.78	26.73	26.85	1730797477.445	20.32	860759294.213	20.96
29.32	29.25	29.43	1574304651.412	18.48	565987711.008	13.78
13.84	13.8	13.9	253463270.579	100.0	128324436.842	100.0
15.78	15.73	15.84	272322657.117	23.37	135478813.09	24.01
18.62	18.58	18.64	45463274.44	100.0	26422673.52	100.0
2

The following code allows to calculate the ratio of the analytes with the internal standard. The output is saved as:

***Sample_{n}_row*_ratio.csv**

In [14]:
# Loop over all the designated sample files. Currently works but i am not sure if it might be just exactly the same as the first one, it works but not iterative...
for n in range(sample_nr_low, sample_nr_high):
    divisor_file = f'/content/drive/My Drive/quantitation_output/Sample_{n}_row3.csv'
    directory = '/content/drive/My Drive/quantitation_output'

    if not os.path.isfile(divisor_file):
        print(f"Divisor file does not exist: {divisor_file}")
        continue  # Skip to the next iteration if the divisor file is missing

    divisor_df = pd.read_csv(divisor_file, sep=',', header=None)
    csv_files = glob.glob(os.path.join(directory, f'*{n}_row*.csv'))

    # Process each CSV file
    for csv_file in csv_files:
        if csv_file == divisor_file or '_ratio' in csv_file:
            continue  # Skip the divisor file itself and ratio files
        data_df = pd.read_csv(csv_file, sep=',', header=None)
        data_df = data_df.apply(pd.to_numeric, errors='coerce')  # Convert values to numeric
        print('divisor_df:', divisor_df)
        print('data_df:', data_df)
        divided_data_df = pd.DataFrame()

        for i in range(len(data_df)):
          # Perform division on the data on the complete dataframe
          divided_row = data_df.iloc[i] / divisor_df.iloc[i]
          divided_data_df = divided_data_df.append(divided_row, ignore_index=True)

        # Create a new file name with the "_ratio" tag
        new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv'

        divided_data_df.to_csv(new_file_name, sep=',', header=False, index=False)
        control_import_df = pd.read_csv(new_file_name, sep=',', header=None)

        # Compare the recalculated values with the content of the ratio file
        """if control_import_df.equals(divided_data_df):
            print(f"The files match: {new_file_name}")
        else:
            print(f"The files do not match: {new_file_name}")"""

        #print(control_import_df)
        print(divided_data_df)
        print(csv_file)
        print('****************************************************')
    print('===========================================================')

divisor_df:        0      1     2             3      4             5      6
0  18.63  18.57  18.7  6.389608e+09  100.0  3.088648e+09  100.0
data_df:        0      1     2             3      4             5      6
0  13.84  13.79  13.9  1.439528e+10  100.0  7.362408e+09  100.0
          0         1         2        3    4         5    6
0  0.742888  0.742596  0.743316  2.25292  1.0  2.383699  1.0
/content/drive/My Drive/quantitation_output/Sample_1_row1.csv
****************************************************
divisor_df:        0      1     2             3      4             5      6
0  18.63  18.57  18.7  6.389608e+09  100.0  3.088648e+09  100.0
data_df:        0      1      2             3      4             5      6
0  15.78  15.73  15.83  1.575032e+10  14.16  7.946364e+09  15.56
          0         1         2        3       4         5       6
0  0.847021  0.847065  0.846524  2.46499  0.1416  2.572764  0.1556
/content/drive/My Drive/quantitation_output/Sample_1_row2.csv
***********

<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce4

          0         1         2         3    4         5    6
0  0.742888  0.742196  0.744111  2.917086  1.0  2.906656  1.0
/content/drive/My Drive/quantitation_output/Sample_2_row1.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.63  18.58  18.68  5.372215e+08  100.0  2.658705e+08  100.0
data_df:        0      1      2             3      4             5      6
0  15.77  15.72  15.84  1.684354e+09  19.77  8.427851e+08  20.52
          0         1         2         3       4         5       6
0  0.846484  0.846071  0.847966  3.135306  0.1977  3.169908  0.2052
/content/drive/My Drive/quantitation_output/Sample_2_row2.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.63  18.58  18.68  5.372215e+08  100.0  2.658705e+08  100.0
data_df:        0      1     2             3      4             5     6
0  20.64  20.59

<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce4

divisor_df:        0      1      2            3      4            5      6
0  18.62  18.58  18.64  45463274.44  100.0  26422673.52  100.0
data_df:        0      1      2             3      4             5      6
0  20.63  20.58  20.68  3.698002e+08  31.74  1.914509e+08  33.92
          0         1         2         3       4         5       6
0  1.107948  1.107643  1.109442  8.134042  0.3174  7.245707  0.3392
/content/drive/My Drive/quantitation_output/Sample_3_row4.csv
****************************************************
divisor_df:        0      1      2            3      4            5      6
0  18.62  18.58  18.64  45463274.44  100.0  26422673.52  100.0
data_df:        0      1     2             3     4             5    6
0  25.64  25.57  25.7  1.148496e+08  9.86  5.471688e+07  9.7
          0         1         2         3       4        5      6
0  1.377014  1.376211  1.378755  2.526205  0.0986  2.07083  0.097
/content/drive/My Drive/quantitation_output/Sample_3_row5.csv
*********

<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-14-ae8ce48c7609>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)


The following code takes the ratio files as well as the parameters for the linear regression (analyte specific), and yields a final output file that contains all the information, such as the information of the ratio used for the quantitaiton, the parameters for the linear equation as well as other information such as compound id and other general sample information.

The final outpur files have the following name format:

***Sample_{n}_quantitation_output.csv***


In [17]:
current_date = datetime.today().strftime('%Y-%m-%d')
output_dir = '/content/drive/My Drive/quantitation_export/'
#header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]
header_row2 = ['Analyte', 'Intercept', 'Slope', 'Value', 'Concentration [microg/ml]']

row_descriptors = ['place holder','Dimethylphthalate', 'Diethylphthalate', 'Internal standard', 'Dibutylphthalate', 'Benzylbutylphthalate', 'Bis(2-ethyl-hexyl)phthalate', 'Di-n-octylphthalate']
file_pattern = '/content/drive/My Drive/quantitation_output/Sample_{}_row{}_ratio.csv'

for n in range(sample_nr_low, sample_nr_high):
    output_file = output_dir + f'Sample_{n}_concentration_output.csv'
    header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header_row1)
        writer.writerow(header_row2)

        for i in range(1, 8):
            if i == 3:
                continue
            print(f'Sample nr: {n}')
            print(f'Compound iteration: {i}')
            linreg_df = pd.read_csv(f'/content/drive/My Drive/calibration_export/Cal_{i}_analyte_linreg.csv')
            print(f'linear function information: {linreg_df}')
            descriptor = row_descriptors[i]
            print(f'Descriptor: {descriptor}')
            file_path = file_pattern.format(n, i)
            if not os.path.isfile(file_path):
                print(f"File not found for Sample {n}, Row {i}")
                continue

            sample_df = pd.read_csv(file_path, header=None, delimiter=',')
            print(f'Sample dataframe: {sample_df}')

            # Extracting the slope value from the linear function information
            intercept_str = str(linreg_df['Intercept'].values[0])
            slope_str = linreg_df['Slope'].values[0]

            # Remove the brackets before using regular expressions
            slope_str_cleaned = slope_str.strip('[]')
            slope_value = ast.literal_eval(slope_str_cleaned)
            if isinstance(slope_value, list):
                slope_value = slope_value[0]  # If slope is a list, take the first element

            intercept = float(re.findall(r'[-\d.]+', intercept_str)[0])
            slope = float(slope_value)

            fourth_column = sample_df.iloc[:, 3]  # Accessing the 4th column
            value = float(fourth_column.iloc[0])  # Assuming you want the value from the first row (index 0)
            concentration = slope * value + intercept
            print(f"Calculated concentration: {concentration}")
            row_data = [descriptor, intercept, slope, value, concentration]
            writer.writerow(row_data)

            # Additional print statements for debugging
            print(f"Sample DataFrame shape: {sample_df.shape}")
            print(f"Linreg DataFrame shape: {linreg_df.shape if linreg_df is not None else None}")
            print("------------------------------------------------------------------------")



Sample nr: 1
Compound iteration: 1
linear function information:    Intercept         Slope  R2 Value
0  -0.015126  [0.31804656]  0.991129
Descriptor: Dimethylphthalate
Sample dataframe:           0         1         2        3    4         5    6
0  0.742888  0.742596  0.743316  2.25292  1.0  2.383699  1.0
Calculated concentration: 0.7014077044720457
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
------------------------------------------------------------------------
Sample nr: 1
Compound iteration: 2
linear function information:    Intercept         Slope  R2 Value
0  -0.018923  [0.35295079]  0.990664
Descriptor: Diethylphthalate
Sample dataframe:           0         1         2        3       4         5       6
0  0.847021  0.847065  0.846524  2.46499  0.1416  2.572764  0.1556
Calculated concentration: 0.8510973581146807
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
------------------------------------------------------------------------
Sample nr: 1
